# Recherche et alignement : outil de recherche iconographique dans la base de données Gallica, Joconde et Europeana

Cet outil permet la récupération de données (textuelles et iconographiques) à partir de termes de recherche de la bases de données : Gallica, Joconde et Europeana

*Document d'entrée* : mot-clé.

*Documents de sortie* : Dans le dossier de travail, les fichiers de données ainsi que les fichiers téléchargés

In [ ]:
# Connexion du notebook à son compte Google Drive et signalement du dossier de travail :

"""
Google Colab notebook.
Python == 3.7.11

Marina Giardinetti
2024
"""

## Installation des bibliothèques et connexion au compte Google Drive

# Désinstallation et installation spécifique de pyOpenSSL
!pip uninstall pyOpenSSL==0.15.1 -y
!pip install pyOpenSSL==17.2.0

# Installation des bibliothèques nécessaires
!pip install utils soup2dict langdetect xmltojson unidecode==1.2.0 folium

# Importation des bibliothèques
from google.colab import drive
import os

# Connexion à Google Drive
drive.mount('/content/drive/')

# Indiquer le chemin vers le dossier de travail sur le Google Drive
work_dir = '/content/drive/My Drive/API/' # @param {type:"string"}

# Création du dossier de travail s'il n'existe pas encore
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
os.chdir(work_dir)

# Configuration des en-têtes et définition du terme à chercher
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
search_term = "cleopatre" # @param {type:"string"}

# Création et changement de répertoire pour le terme à chercher
os.makedirs(search_term, exist_ok=True)
os.chdir(search_term)

# Chemin du dossier
dir_path = os.path.join(work_dir, search_term)


# Gallica

In [4]:
import os
import re
import time
import requests
from collections import OrderedDict
from bs4 import BeautifulSoup
import xmltodict
import pandas as pd

# Configuration du chemin de travail
os.chdir(dir_path)
dossier_gallica = 'images_gallica'
os.makedirs(dossier_gallica, exist_ok=True)
donnees_gallica = {}

# Création du lien de recherche Gallica
if ' ' in search_term:
    nom_lien = search_term.replace(' ', '%20')
else:
    nom_lien = search_term
url_gallica = f"https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=dc.title%20adj%20{nom_lien}&maximumRecords=49&deway=%20any%207"

# Requête à Gallica
reponse = requests.get(url_gallica, verify=False, headers=headers, timeout=5)

# Parsing de la réponse XML
parse = BeautifulSoup(reponse.content, 'html.parser')
with open('temp.xml', 'w') as fichier_temporaire:
    fichier_temporaire.write(parse.prettify())
with open('temp.xml', 'r') as fichier_temporaire:
    gallica_dict = xmltodict.parse(fichier_temporaire.read())
os.remove('temp.xml')

# Extraction des données
for record in gallica_dict['srw:searchretrieveresponse']['srw:records']['srw:record']:
    titre = ''
    date = ''
    auteur = ''
    types = ''
    sujet = ''
    lien_image = ''
    for cle, valeur in record.items():
        if cle == 'srw:recorddata':
            for cle2, valeur2 in valeur.items():
                for cle3, valeur3 in valeur2.items():
                    if cle3 == 'dc:title':
                        titre = re.sub(r"[][]", '', str(valeur3))
                        donnees_gallica[titre] = {}
                    if cle3 == 'dc:date':
                        date = valeur3
                    if cle3 == 'dc:creator':
                        auteur = valeur3
                    if cle3 == 'dc:type':
                        types = valeur3
                    if cle3 == 'dc:subject':
                        sujet = valeur3
                    if cle3 == 'dc:identifier':
                        identifiants = valeur3 if isinstance(valeur3, list) else [valeur3]
                        for identifiant in identifiants:
                            id_match = re.findall('b[a-z0-9]{4,}', str(identifiant))
                            for identifiant_trouve in id_match:
                                lien_image = f'http://gallica.bnf.fr/iiif/ark:/12148/{identifiant_trouve}/f1/full/3000/0/native.jpg'
                                chemin_image = os.path.join(chemin_dossier, dossier_gallica, f'{identifiant_trouve}.jpg')
                                time.sleep(15)
                                requete_image = requests.get(lien_image, headers=headers, stream=True)
                                with open(chemin_image, 'wb') as f:
                                    f.write(requete_image.content)
    donnees_gallica[titre] = {'Date': date, 'Auteur': auteur, 'Type': types, 'Sujet': sujet, 'Lien': lien_image}

# Conversion en DataFrame et exportation en Excel
df_gallica = pd.DataFrame.from_dict(donnees_gallica, orient='index')
df_gallica.to_excel('donnees_gallica.xlsx')


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gallica.bnf.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


# Europeana

In [6]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from langdetect import detect, LangDetectException
import ssl
from urllib.error import HTTPError
import urllib

# Configuration du chemin de travail
os.chdir(dir_path)
europeana_images_dir = 'europeana_images'
os.makedirs(europeana_images_dir, exist_ok=True)
europeana_data = {}

# Création du lien de recherche Europeana
search_term_encoded = search_term.replace(' ', '%20') if ' ' in search_term else search_term
europeana_search_url = f"https://api.europeana.eu/record/v2/search.json?query={search_term_encoded}&rows=100&theme=art&wskey=alemalle"
search_request = urllib.request.Request(europeana_search_url)

try:
    search_response = urllib.request.urlopen(search_request)
    parsed_response = BeautifulSoup(search_response, 'html.parser')
    search_results = json.loads(parsed_response.text)
except HTTPError:
    print('Pas de téléchargement possible pour la base Europeana avec cette recherche')
    search_results = {}

# Désactiver les vérifications SSL
ssl._create_default_https_context = ssl._create_unverified_context

# Extraction des données
if 'items' in search_results:
    for item in search_results['items']:
        title = ''
        authors = ''
        item_type = ''
        subjects = ''
        image_link = ''

        if 'title' in item:
            title = item['title'][0] if isinstance(item['title'], list) else item['title']
            cleaned_title = re.sub(r"[\['\]]", '', title)

        if 'dcCreator' in item:
            authors = str(item['dcCreator'])

        if 'edmConceptPrefLabelLangAware' in item and 'fr' in item['edmConceptPrefLabelLangAware']:
            item_type = str(item['edmConceptPrefLabelLangAware']['fr'])

        if 'edmConceptLabel' in item:
            for concept in item['edmConceptLabel']:
                for value in concept.values():
                    try:
                        if detect(value) == 'fr':
                            subjects = str(value)
                    except LangDetectException:
                        pass

        if 'edmIsShownBy' in item:
            for link in item['edmIsShownBy']:
                image_filename = os.path.join(work_dir, search_term, europeana_images_dir, re.sub('\/', '_', cleaned_title) + '.jpg')
                if 'gallica' not in link:
                    try:
                        response = requests.get(link, headers=headers, stream=True)
                        with open(image_filename, 'wb') as image_file:
                            image_file.write(response.content)
                    except (NameError, requests.exceptions.SSLError, OSError):
                        pass

        europeana_data[cleaned_title] = {'Auteur': authors, 'Type': item_type, 'Sujet': subjects, 'Lien': link}

# Conversion des données en DataFrame et exportation en fichier Excel
europeana_df = pd.DataFrame.from_dict(europeana_data, orient='index')
europeana_df.to_excel('europeana_data.xlsx')


# Joconde
Téléchargement des métadonnées

In [9]:
import os
import re
import json
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

# Configuration du chemin de travail
os.chdir(dir_path)
joconde_data = {}

# Création du lien de recherche Joconde
search_term_encoded = search_term.replace(' ', '%20')
joconde_search_url = f"https://data.culture.gouv.fr/api/records/1.0/search/?dataset=base-joconde-extrait&q={search_term_encoded}&rows=10000"

# Requête à Joconde
response = urllib.request.urlopen(joconde_search_url)
soup = BeautifulSoup(response, 'html.parser')
joconde_records = json.loads(soup.text)

# Extraction des données
for record in joconde_records.get('records', []):
    fields = record.get('fields', {})
    title = fields.get('titre', '')
    author = fields.get('auteur', '')
    materials = fields.get('materiaux_techniques', '')
    subject = fields.get('sujet_represente', '')
    date = fields.get('epoque', fields.get('periode_de_creation', ''))

    joconde_data[title] = {
        'Date': date,
        'Auteur': author,
        'Matériaux': materials,
        'Sujet': subject
    }

# Conversion des données en DataFrame et exportation en fichier Excel
joconde_df = pd.DataFrame.from_dict(joconde_data, orient='index')
joconde_df.to_excel('donnees_joconde.xlsx')
